### Goal

Balance split of the data. Save into files the individuals for train, test and valid.

###  TODO
- split dataset by individuals and taking care to have a balanced set in terms of #samples (some invidivuals tent to have more samples as the others), by gender and by WGS/WES

### Conclusions

Under `/s/project/rep/processed/gtex/recount` are the list of individuals for train, valid, test.

`train_individuals.txt`,`valid_individuals.txt`, `test_individuals.txt` - list of individuals

`train_individuals.csv`,`valid_individuals.csv`, `test_individuals.csv` - description of samples for these individuals

In [2]:
import os
import sys

import pandas as pd
import numpy as np

import rep.preprocessing as p
from rep.constants import ANNDATA_CST as a

%aimport

Modules to reload:
all-except-skipped

Modules to skip:



In [9]:
file = os.path.join(os.readlink(os.path.join("..","..","data")),"processed","gtex","recount","recount_gtex.h5ad")
# gtex = p.load(file)
gtex = p.load('/data/nasif12/home_if12/giurgiu/rep_gagneur/rep/notebooks/preprocessing/tmp1547561549.h5ad')

In [11]:
%time (train,valid,test) = p.split_by_individuals(gtex,groupby=['Gender','Indiv_Seq_Assay'])
"Individual split: ", len(train),len(valid),len(test)

Total individuals: 555
Individual split before balancing:  335 113 107
Iteration:  1
	Expc counts:  [5797, 1932, 1932]
	Real counts:  6499 1912 1334
	Expc fraction:  [0.6, 0.2, 0.2]
	Real fraction:  0.6726350652038915 0.1978886358931898 0.13806665286690126
	Diff:  [0.07263506520389151, -0.002111364106810204, -0.061933347133098754]
remove best
Min error:  0.06207824466984069
K-562
Iteration:  2
	Expc counts:  [5797, 1932, 1932]
	Real counts:  6397 2014 1334
	Expc fraction:  [0.6, 0.2, 0.2]
	Real fraction:  0.6620782446698407 0.20844545642724074 0.13806665286690126
	Diff:  [0.06207824466984069, 0.008445456427240733, -0.061933347133098754]
remove best
Min error:  0.0577313185675844
GTEX-NPJ8
remove best
Min error:  0.002235561995446056
K-562
Iteration:  3
	Expc counts:  [5797, 1932, 1932]
	Real counts:  6355 1954 1436
	Expc fraction:  [0.6, 0.2, 0.2]
	Real fraction:  0.6577313185675844 0.20223556199544607 0.1486234734009522
	Diff:  [0.0577313185675844, 0.002235561995446056, -0.05137652659

('Individual split: ', 317, 115, 123)

#### Check the uniformity of the datasets

In [33]:
samples_info = gtex.var
samples_info[['Gender','Individual']].groupby('Gender').size()

Gender
           756
female    3375
male      5531
dtype: int64

In [13]:
df_train = samples_info[samples_info['Individual'].isin(train)]
df_valid = samples_info[samples_info['Individual'].isin(valid)]
df_test = samples_info[samples_info['Individual'].isin(test)]

- check woman and man ratio

In [29]:
states = ['train','valid','test']
dict_states = {'train':df_train,'valid':df_valid,'test':df_test}
dict_states_indiv = {'train':train,'valid':valid,'test':test}
total_len = len(train) + len(test) + len(valid)

print("Expected split: [0.6, 0.2, 0.2]")

for s in states: print(s," #individuals = ", len(dict_states_indiv[s]), " fraction = " ,(len(dict_states_indiv[s])/total_len))

print()
for s in states: print(s," #samples = ", dict_states[s].shape, " fraction = ", dict_states[s].shape[0]/samples_info.shape[0])

print()
for s in states:
    df = dict_states[s]
    l = dict_states_indiv[s]
    print(s," female = ", (df[df['Gender'] == 'female'][['Individual','Gender']].drop_duplicates().shape[0] / len(l)), "male = ", (df[df['Gender'] == 'male'][['Individual','Gender']].drop_duplicates().shape[0] / len(l)))

print()
for s in states:
    df = dict_states[s]
    l = dict_states_indiv[s]
    print(s," WGS = ", (df[df['Indiv_Seq_Assay'] == 'WGS'][['Individual','Indiv_Seq_Assay']].drop_duplicates().shape[0] / len(l)), "WES = ", (df[df['Indiv_Seq_Assay'] == 'WES'][['Individual','Indiv_Seq_Assay']].drop_duplicates().shape[0] / len(l))) 

Expected split: [0.6, 0.2, 0.2]
train  #individuals =  317  fraction =  0.5711711711711712
valid  #individuals =  115  fraction =  0.2072072072072072
test  #individuals =  123  fraction =  0.22162162162162163

train  #samples =  (5803, 14)  fraction =  0.6006002897950735
valid  #samples =  (1942, 14)  fraction =  0.20099358310908713
test  #samples =  (1941, 14)  fraction =  0.20089008486855722

train  female =  0.3280757097791798 male =  0.5709779179810726
valid  female =  0.3130434782608696 male =  0.591304347826087
test  female =  0.3252032520325203 male =  0.5934959349593496

train  WGS =  0.9495268138801262 WES =  0.04416403785488959
valid  WGS =  0.9304347826086956 WES =  0.06086956521739131
test  WGS =  0.959349593495935 WES =  0.04065040650406504


In [38]:
# save individuals
path = os.path.join(os.readlink(os.path.join("..","..","data")),"processed","gtex","recount")
for s in states:
    with open(os.path.join(path,s + "_individuals.txt"), 'w') as f: 
        for item in dict_states_indiv[s]:
            f.write("%s\n" % item)

In [39]:
# save data information per training
for s in states:
    file = os.path.join(path,s + "_individuals.csv")
    dict_states[s].to_csv(file, encoding='utf-8', sep="\t")